In [3]:
# Fuel Pricing Model



import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from datetime import date, timedelta



df = pd.read_csv('Nat_Gas.csv')



df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')

start_date = df['Dates'].min()

df['Days'] = (df['Dates'] - start_date).dt.days



df['sin_time'] = np.sin(2 * np.pi * df['Days'] / 365.25)

df['cos_time'] = np.cos(2 * np.pi * df['Days'] / 365.25)



X = df[['Days', 'sin_time', 'cos_time']]

y = df['Prices']



model = LinearRegression()

model.fit(X,y)



def gas_price(date):

    corrected_date = pd.to_datetime(date)

    days = (corrected_date - start_date).days



    sin_val = np.sin(2 * np.pi * days / 365.25)

    cos_val = np.cos(2 * np.pi * days / 365.25)



    indep_vars_df = pd.DataFrame([[days, sin_val, cos_val]], columns = ['Days', 'sin_time', 'cos_time'])

    prediction = model.predict(indep_vars_df)

    

    return prediction[0]





# 'rate' refers to the rate at which the fuel can be injected or withdrawed at (vol/day)

# storage_cost_rate: Cost to store gas per unit per month

# injection_withdrawal_cost_rate: Cost to inject/withdraw per unit of gas



def contract_price(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, total_vol, injection_withdrawal_cost_rate):   



    buy_cost = 0

    cash_in = 0

    revenue = 0

    

    for i, date in enumerate(in_dates):

        if in_prices is not None and len(in_prices) > i: # If in_prices is NOT None AND the list has an element at index i, use the provided price.

            price = in_prices[i]

        else:

            price = gas_price(date)



        amount_to_inject = total_vol / len(in_dates)



        buy_cost += price * amount_to_inject   # The price is per unit vol so we multiply by the volume we inject at each date

        injection_cost = amount_to_inject * injection_withdrawal_cost_rate

        buy_cost += injection_cost



    for i, date in enumerate(out_dates):

        if out_prices is not None and len(out_prices) > i: # If out_prices is NOT None AND the list has an element at index i, use the provided price.

            price = out_prices[i]

        else:

            price = gas_price(date)



        amount_to_withdraw = total_vol / len(out_dates)

        revenue += price * amount_to_withdraw

        withdrawal_cost = amount_to_withdraw * injection_withdrawal_cost_rate

        cash_in += revenue - withdrawal_cost

    
    start_dt = pd.to_datetime(in_dates[0])

    end_dt = pd.to_datetime(out_dates[-1])

    months_held = (end_dt - start_dt).days / 30.44 # Average days in a month

    total_storage_cost = storage_cost_rate * total_vol * months_held



    # Final value

    contract_value = cash_in - buy_cost - total_storage_cost

    return contract_value



inputs = {

    "in_dates": ['2023-06-15'],  # Buy in Summer

    "in_prices": None,           # Let the model predict price

    "out_dates": ['2023-12-15'], # Sell in Winter

    "out_prices": None,          # Let the model predict price

    "rate": 100000,              # Max injection rate (not fully enforced in prototype)

    "storage_cost_rate": 0.10,   # $0.10 per unit per month (e.g. $100k/month for 1M units)

    "total_vol": 1000000,        # 1 Million MMBtu

    "injection_withdrawal_cost_rate": 0.01 # $0.01 per unit to move gas

}



# Run the pricing

value = contract_price(**inputs)